In [5]:
import eeg_decoder 
import pandas as pd
import numpy as np
import scipy.io as io
from pathlib import Path
import shutil

In [6]:
experiment_name = 'd01'
data_dir = f'./data/{experiment_name}'
exp = eeg_decoder.Experiment(experiment_name, data_dir, dev=False,info_from_file=False)
sub_folders = [f for f in Path(exp.data_dir).iterdir() if f.is_dir()][:-2]
sub_folders

[WindowsPath('data/d01/04'),
 WindowsPath('data/d01/06'),
 WindowsPath('data/d01/07'),
 WindowsPath('data/d01/08'),
 WindowsPath('data/d01/09'),
 WindowsPath('data/d01/10'),
 WindowsPath('data/d01/11'),
 WindowsPath('data/d01/12'),
 WindowsPath('data/d01/13'),
 WindowsPath('data/d01/14'),
 WindowsPath('data/d01/16'),
 WindowsPath('data/d01/17'),
 WindowsPath('data/d01/19'),
 WindowsPath('data/d01/20'),
 WindowsPath('data/d01/21'),
 WindowsPath('data/d01/22'),
 WindowsPath('data/d01/23'),
 WindowsPath('data/d01/24'),
 WindowsPath('data/d01/25'),
 WindowsPath('data/d01/26')]

In [13]:
for isub, sub_folder in enumerate(sub_folders):

    exp.xdata_files = list(sub_folder.rglob('*xdata.mat'))
    exp.ydata_files = list(sub_folder.rglob('*ydata.mat'))
    exp.behavior_files = list(sub_folder.rglob('*behavior.csv'))
    exp.artifact_idx_files = list(sub_folder.rglob('*artifact_idx.mat'))        

    # load and concatenate data from behavior, eeg, and trial labels
    for irun in range(len(exp.xdata_files)): 
        x1, y1 = exp.load_eeg(irun)
        idx1 = exp.load_artifact_idx(irun)
        beh1 = exp.load_behavior(irun, remove_artifact_trials=False)

        if irun == 0:
            x = x1 
            y = y1
            idx = idx1
            beh = pd.DataFrame(beh1)
            # sub_num = beh.loc[0,'Subject']
            sub_num = exp.xdata_files[0].stem[4:6]
        else:
            x = np.concatenate((x, x1), 0)
            y = np.concatenate((y, y1), 0)
            idx = np.concatenate((idx, idx1), 0)
            beh1_df = pd.DataFrame(beh1)
            beh = pd.concat((beh, beh1_df))

    x = np.moveaxis(x, 0, 2) # roll bc saving with savemat changes shape for some reason...

    # save
    io.savemat(f'data/d01/d01_{sub_num}_xdata.mat',{'xdata':x})
    io.savemat(f'data/d01/d01_{sub_num}_ydata.mat',{'ydata':y})
    io.savemat(f'data/d01/d01_{sub_num}_artifact_idx.mat',{'artifact_idx':idx})
    beh.to_csv(f'data/d01/d01_{sub_num}_behavior.csv')
    shutil.copy(f'data/d01/{sub_num}/d01_{sub_num}_1_info.mat',f'data/d01/d01_{sub_num}_info.mat')

In [ ]:
for isub, sub_folder in enumerate(sub_folders):
    sub_num = sub_folder.name
    shutil.copy(f'data/d01/{sub_num}/d01_{sub_num}_1_info.mat',f'data/d01/d01_{sub_num}_info.mat')